In [1]:
from dotenv import load_dotenv
from sqlalchemy.testing.suite.test_reflection import metadata

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH11-Retriever")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH11-Retriever


In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

In [4]:
print(docs[5].page_content[:500])

1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언
n 영국 블레츨리 파크에서 개최된 AI 안전성 정상회의에 참가한 28개국들이 AI 안전 보장을 
위한 협력 방안을 담은 블레츨리 선언을 발표
n 첨단 AI를 개발하는 국가와 기업들은 AI 시스템에 대한 안전 테스트 계획에 합의했으며, 
영국의 AI 안전 연구소가 전 세계 국가와 협력해 테스트를 주도할 예정 
KEY Contents
£ AI 안전성 정상회의 참가국들, 블레츨리 선언 통해 AI 안전 보장을 위한 협력에 합의
n 2023년 11월 1~2일 영국 블레츨리 파크에서 열린 AI 안전성 정상회의(AI Safety Summit)에 
참가한 28개국 대표들이 AI 위험 관리를 위한 ‘블레츨리 선언’을 발표 
∙선언은 AI 안전 보장을 위해 국가, 국제기구, 기업, 시민사회, 학계를 포함한 모든 이해관계자의 협력이 
중요하다고 강조했으며,


# Chunk + 원본 문서 검색

In [5]:
import uuid
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever

vectorstore = Chroma(
    collection_name="small_bigger_chunks",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

# 부모 문서의 저장소 계층
store = InMemoryStore()

id_key = "doc_id"

# 검색기(시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    id_key=id_key,
    byte_store=store,
)

# 문서 ID를 생성
doc_ids = [str(uuid.uuid4()) for _ in docs]

# 두 개의 생성된 id를 확인
doc_ids

['cb0f49c0-ad7b-4a7e-87c0-f3680b766ecf',
 'aa6c7faa-f109-4a08-bb58-6f5053b47780',
 '68623475-2814-415a-a238-ce3fa2074abc',
 'e3695dde-ce81-4600-b428-8c692098a913',
 'a823de65-123d-4adb-812c-1fd7e722960b',
 'c89298df-536b-4b56-9260-20d6f7c1a01e',
 'b20e116e-8f20-46cd-a561-0ef0d0d26afb',
 '35af4e77-9bbb-454c-b9a7-9acc44c107a7',
 '593e692d-297c-4234-82a7-082e54dcc37e',
 '27e4706b-4664-4462-b06f-92270981aeae',
 '33efaf30-e152-4fd7-95ea-cc36897c6fad',
 '22c7fe8d-2dab-44fe-83e5-f4de2897d38a',
 'eb1abb73-c6c7-4836-942e-ad3e300d9181',
 'c05bad8a-8a0a-404a-9e1e-96024cf8b771',
 'f65f60fd-7faa-46a5-8bbe-f98f2081a6aa',
 '922f3c20-cd0d-4df0-a3b0-d2d1031b5a0c',
 '92a61134-17a6-45c9-b72a-749d68c155e4',
 '5d60c15d-5f84-49b8-9ab8-34551dc34108',
 'fbb552c2-e2a0-42a7-93e3-58f1a32a6489',
 'a4ded913-5efa-4de9-b76a-90c0a3ecc1ba',
 '0052d245-0e28-43f4-8e85-9894e6a557b1',
 'b1c785db-5c2f-4bfa-888c-2c17a829cf14',
 '9e38a263-93c9-4d7b-b784-f1169764a422']

In [6]:
# RecursiveCharacterTextSplitter 객체를 생성
parent_text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)

# 더 작은 청크를 생성하는 데 사용할 분할기
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

In [7]:
parent_docs = []

for i, doc in enumerate(docs):
    # 현재 문서의 ID를 가져옴
    _id = doc_ids[i]
    # 현재 문서를 하위 문서로 분할
    parent_doc = parent_text_splitter.split_documents([doc])

    for _doc in parent_doc:
        # metadata에 문서 ID를 저장
        _doc.metadata[id_key] = _id
    parent_docs.extend(parent_doc)

In [10]:
parent_docs[0].metadata

{'producer': 'Hancom PDF 1.3.0.542',
 'creator': 'Hwp 2018 10.0.0.13462',
 'creationdate': '2023-12-08T13:28:38+09:00',
 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'moddate': '2023-12-08T13:28:38+09:00',
 'trapped': '',
 'modDate': "D:20231208132838+09'00'",
 'creationDate': "D:20231208132838+09'00'",
 'page': 0,
 'doc_id': 'cb0f49c0-ad7b-4a7e-87c0-f3680b766ecf'}

In [11]:
child_docs = []
for i, doc in enumerate(docs):
    # 현재 문서의 ID를 가져옴
    _id = doc_ids[i]
    # 현재 문서의 하위 문서로 분할
    child_doc = child_text_splitter.split_documents([doc])
    for _doc in child_doc:
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)

In [12]:
# 생성된 Child 문서의 메타데이터를 확인
child_docs[0].metadata

{'producer': 'Hancom PDF 1.3.0.542',
 'creator': 'Hwp 2018 10.0.0.13462',
 'creationdate': '2023-12-08T13:28:38+09:00',
 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'moddate': '2023-12-08T13:28:38+09:00',
 'trapped': '',
 'modDate': "D:20231208132838+09'00'",
 'creationDate': "D:20231208132838+09'00'",
 'page': 0,
 'doc_id': 'cb0f49c0-ad7b-4a7e-87c0-f3680b766ecf'}

In [13]:
print(f"분할된 parent_docs의 개수: {len(parent_docs)}")
print(f"분할된 child_docs의 개수: {len(child_docs)}")

분할된 parent_docs의 개수: 73
분할된 child_docs의 개수: 440


In [14]:
# 백터 저장소에 parent + child 문서를 추가
retriever.vectorstore.add_documents(parent_docs)
retriever.vectorstore.add_documents(child_docs)

# docstore에 원본 문서를 저장
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [15]:
# vectorstore의 유사도 검색을 수행
relevant_chunks = retriever.vectorstore.similarity_search("삼성전자가 만든 생성형 AI의 이름은?")
print(f"검색된 문서의 개수: {len(relevant_chunks)}")

검색된 문서의 개수: 4


In [16]:
for chunk in relevant_chunks:
    print(chunk.page_content, end="\n\n")
    print(">" * 100, end="\n\n")

☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.
삼성전자, ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10
   ▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 

In [17]:
relevant_docs = retriever.invoke("삼성전자가 만든 생성형 AI의 이름은?")
print(f"검색된 문서의 개수: {len(relevant_docs)}", end="\n\n")
print("=" * 100, end="\n\n")
print(relevant_docs[0].page_content)

검색된 문서의 개수: 2


SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 
2024년부터 가우스를 탑재한 삼성

In [18]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 MMR(Maximal Marginal Relevvance)로 설정
retriever.search_type = SearchType.mmr

# 관련 문서 전체를 검색
print(retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")[0].page_content)

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 
2024년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Ll

In [19]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 similarity로 설정, k값을 1로 설정
retriever.search_type = SearchType.similarity
retriever.search_kwargs = {"k": 1}

# 관련 문서 전체를 검색
print(len(retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")))

1


# 요약본(summary)을 벡터저장소에 저장

In [20]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)

# PDF 파일 로드 및 텍스트 분할 실행
split_docs = loader.load_and_split(text_splitter)

# 분할된 문서의 개수 출력
print(f"분할된 문서의 개수: {len(split_docs)}")

분할된 문서의 개수: 61


In [21]:
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

summary_chain = (
    {"doc": lambda x: x.page_content}
    # 문서 요약을 위한 프롬프트 템플릿 생성
    | ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert in summarizing documents in Korean."),
            (
                "user",
                "Summarize the following documents in 3 sentences in bullet points format.\n\n{doc}",
            ),
        ]
    )
    # OpenAI의 ChatGPT 모델을 사용하여 요약 생성
    | ChatOpenAI(temperature=0, model="gpt-4o-mini")
    | StrOutputParser()
)

In [22]:
# 문서 배치 처리
summaries = summary_chain.batch(split_docs, {"max_concurrency": 10})

In [23]:
len(summaries)

61

In [24]:
# 원본 문서의 내용을 출력합니다.
print(split_docs[33].page_content, end="\n\n")
# 요약을 출력합니다.
print("[요약]")
print(summaries[33])

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에

[요약]
- 삼성전자가 온디바이스에서 작동 가능한 생성 AI 모델 '삼성 가우스'를 공개하였으며, 이는 언어, 코드, 이미지의 3개 모델로 구성되어 있다.
- '삼성 가우스'는 정규분포 이론을 정립한 수학자 가우스의 이름을 따왔으며, 다양한 상황에 최적화된 모델 선택이 가능하다.
- 삼성전자는 이 AI 모델이 사용자 정보를 외부로 유출하지 않도록 설계되었으며, 향후 다양한 제품에 단계적으로 탑재할 계획이다.


In [33]:
import uuid

# 요약 정보를 저장할 벡터 저장소를 생성
summary_vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

# 부모 문서를 저장할 저장소를 새엇ㅇ
store = InMemoryStore()

# 문서 ID를 저장할 키 이름을 지정
id_key = "doc_id"

# 검색기를 초기화
retriever = MultiVectorRetriever(
    vectorstore=summary_vectorstore,
    id_key=id_key,
    byte_store=store,
)

# 문서 ID를 생성
doc_ids = [str(uuid.uuid4()) for _ in split_docs]

In [35]:
summary_docs = [
    # 요약된 내용을 페이지 콘텐츠로 하고, 문서 ID를 메타데이터로 포함하는 Document 객체를 생성합니다.
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

In [37]:
# 요약본의 문서의 개수
len(summary_docs)

61

In [38]:
retriever.vectorstore.add_documents(summary_docs)

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장
retriever.docstore.mset(list(zip(doc_ids, summary_docs)))

In [39]:
# 유사도 검색을 수행
result_docs = summary_vectorstore.similarity_search("삼성전자가 만든 생성형 AI의 이름은?")

In [40]:
print(result_docs[0].page_content)

- 삼성전자가 온디바이스에서 작동 가능한 생성 AI 모델 '삼성 가우스'를 공개하였으며, 이는 언어, 코드, 이미지의 3개 모델로 구성되어 있다.
- '삼성 가우스'는 정규분포 이론을 정립한 수학자 가우스의 이름을 따왔으며, 다양한 상황에 최적화된 모델 선택이 가능하다.
- 삼성전자는 이 AI 모델이 사용자 정보를 외부로 유출하지 않도록 설계되었으며, 향후 다양한 제품에 단계적으로 탑재할 계획이다.


In [41]:
# 관련된 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("삼성전자가 만든 생성형 AI 의 이름은?")
print(retrieved_docs[0].page_content)

- 삼성전자가 온디바이스에서 작동 가능한 생성 AI 모델 '삼성 가우스'를 공개하였으며, 이는 언어, 코드, 이미지의 3개 모델로 구성되어 있다.
- '삼성 가우스'는 정규분포 이론을 정립한 수학자 가우스의 이름을 따왔으며, 다양한 상황에 최적화된 모델 선택이 가능하다.
- 삼성전자는 이 AI 모델이 사용자 정보를 외부로 유출하지 않도록 설계되었으며, 향후 다양한 제품에 단계적으로 탑재할 계획이다.


# 가설 쿼리(Hypothetical Queries) 를 활용하여 문서 내용 탐색

In [42]:
functions = [
    {
        "name": "hypothetical_questions",  # 함수의 이름을 지정합니다.
        "description": "Generate hypothetical questions",  # 함수에 대한 설명을 작성합니다.
        "parameters": {  # 함수의 매개변수를 정의합니다.
            "type": "object",  # 매개변수의 타입을 객체로 지정합니다.
            "properties": {  # 객체의 속성을 정의합니다.
                "questions": {  # 'questions' 속성을 정의합니다.
                    "type": "array",  # 'questions'의 타입을 배열로 지정합니다.
                    "items": {
                        "type": "string"
                    },  # 배열의 요소 타입을 문자열로 지정합니다.
                },
            },
            "required": ["questions"],  # 필수 매개변수로 'questions'를 지정합니다.
        },
    }
]

In [43]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_openai import ChatOpenAI

hypothetical_query_chain = (
    {"doc": lambda x: x.page_content}
    # 아래 문서를 사용하여 답변할 수 있는 가상의 질문을 정확히 3개 생성하도록 요청합니다. 이 숫자는 조정될 수 있습니다.
    | ChatPromptTemplate.from_template(
        "Generate a list of exactly 3 hypothetical questions that the below document could be used to answer. "
        "Potential users are those interested in the AI industry. Create questions that they would be interested in. "
        "Output should be written in Korean:\n\n{doc}"
    )
    | ChatOpenAI(max_retries=0, model="gpt-4o-mini").bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    # 출력에서 "questions" 키에 해당하는 값을 추출합니다.
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

In [44]:
# 주어진 문서에 대해 체인을 실행합니다.
hypothetical_query_chain.invoke(split_docs[33])

['삼성 가우스의 온디바이스 작동 기술이 AI 산업에 어떻게 혁신적인 변화를 가져올까요?',
 '삼성전자가 삼성 가우스의 출시를 통해 사용자 개인 정보 보호를 강화할 수 있는 방법은 무엇일까요?',
 '삼성 가우스의 언어, 코드, 이미지 모델이 특정 산업 분야에서 어떤 방식으로 활용될 수 있을까요?']

In [45]:
# 문서 목록에 대해 가설 질문을 배치 생성
hypothetical_questions = hypothetical_query_chain.batch(
    split_docs, {"max_concurrency": 10}
)

In [46]:
hypothetical_questions[33]

['삼성 가우스가 기존 AI 기술과 비교하여 제공할 수 있는 주요 이점은 무엇일까요?',
 '삼성 가우스가 다른 기업의 AI 모델에 비해 사용자 개인 정보를 어떻게 더 안전하게 보호할 수 있을까요?',
 '온디바이스에서 작동하는 삼성 가우스가 소비자 및 기업 사용자에게 가져올 수 있는 잠재적인 변화는 무엇일까요?']

In [47]:
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소
hypothetical_vectorstore = Chroma(
    collection_name="hypo-questions",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

# 부모 문서의 저장소 계층
store = InMemoryStore()

id_key = "doc_id"

# 검색기
retriever = MultiVectorRetriever(
    vectorstore=hypothetical_vectorstore,
    id_key=id_key,
    byte_store=store,
)
doc_ids = [str(uuid.uuid4()) for _ in split_docs]

In [48]:
question_docs = []
# hypothetical_question 저장
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        # 질문 리스트의 각 질문에 대해 Document 객체를 생성하고, 메타데이터에 해당 질문의 문서ID를 포함
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

In [49]:
# hypothetical_questions 문서를 벡터 저장소에 추가
retriever.vectorstore.add_documents(question_docs)

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장
retriever.docstore.mset(list(zip(doc_ids, question_docs)))

In [50]:
# 유사한 문서를 벡터 저장소에서 검색
result_docs = hypothetical_vectorstore.similarity_search(
    "삼성전자가 만든 생성형AI 의 이름은?"
)

In [51]:
# 유사도 검색 결과를 출력
for doc in result_docs:
    print(doc.page_content)
    print(doc.metadata)

삼성전자가 개발한 생성적 AI인 삼성 가우스의 주요 기능과 활용 사례는 무엇인가요?
{'doc_id': 'a8944abf-45e8-42f3-9036-bff86dcc5083'}
삼성전자에서 개발한 생성 AI '삼성 가우스'가 AI 시장에 미칠 잠재적인 영향은 무엇인가?
{'doc_id': 'd219dfc6-6073-4fab-bb69-a60b4ea3830a'}
삼성 가우스가 기존 AI 기술과 비교하여 제공할 수 있는 주요 이점은 무엇일까요?
{'doc_id': 'c2d9ab6f-80db-4b18-b8ac-9b27d6f8147c'}
2023년의 AI 산업 발전을 위해 삼성전자가 어떤 전략을 추진하고 있는지 알 수 있을까요?
{'doc_id': 'a8944abf-45e8-42f3-9036-bff86dcc5083'}


In [52]:
# 관련된 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke(result_docs[1].page_content)

# 검색된 문서를 출력합니다.
for doc in retrieved_docs:
    print(doc.page_content)

AI 윤리에 대한 최근 연구 결과는 어떤 것들이 있으며, 그 의미는 무엇인가?
기후 위기와 세계적 난제를 해결하는 데 AI 기술이 어떻게 기여할 수 있으며, 이를 위한 정책적 지원은 어떤 형태일까?
AI 거버넌스 정책이 효과적으로 구현될 경우, AI 시스템에서 발생할 수 있는 사회적 위험을 어떻게 완화할 수 있을까?
